In [55]:
import torch
import cv2
import numpy as np
from ultralytics import YOLO
import easyocr
import PIL

In [ ]:
yolo_model = YOLO('models/CarNumbers_epoch39.pt')
#yolo_model.eval()

text_model = torch.jit.load('models/scriped_Number_to_Text_ONLY_BIG.pt')
#text_model.eval()

symbols_keys = {
    0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9',
    10: 'А', 11: 'В', 12: 'С', 13: 'Е', 14: 'Н', 15: 'К', 16: 'М', 17: 'О', 18: 'Р', 19: 'Т',
    20: 'Х'
}


In [57]:
def detect_number(image_path):
    image = cv2.imread(image_path)

    #cv2.imshow('i', image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    results = yolo_model.predict(image_rgb, conf=0.5, show = True)

    plates_data = []
    if results:
        boxes = results[0].boxes.xyxy
        #print('\n\n',boxes,'\n\n')

        for box in boxes:
            x1, y1, x2, y2 = box.int()
            plate_image = image[y1:y2, x1:x2]
            #print(plate_image)
            plate_image = cv2.cvtColor(plate_image, cv2.COLOR_BGR2GRAY)
            plate_image_array = plate_image.astype(np.float32)
            plate_image_array /= 255
            plates_data.append(plate_image_array)

    return plates_data
        

# Подсчитываем количество номеров в кадре

In [ ]:
test = detect_number('images-for-tests/Авенсис.jpg')
print('Номеров в кадре:',len(test))
# print(test[1])
cv2.imshow('i', test[0])
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 640x480 1 license-plate, 34.5ms
Speed: 6.9ms preprocess, 34.5ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 480)
Номеров в кадре: 1


# Попробуем с помощью easyocr
Все работает, только буквы нужно сделать заглавными

In [59]:
reader = easyocr.Reader(['ru'])
img = test[0]
img = (img * 255).astype(np.uint8)
result = reader.readtext(img)
for i in result:
    bbox, text, prob = i
    print('_', text)

_ м09ОЕт56


# Здесь контуризация не всегда сработает
Т.к. номер имеет грязь, прилипший мусор и т.д., контуризация выделяет слишком много мусора. Такой подход прокатил бы с текстом в книге или скане документа, но в данном контексте этот способ требует улучшений. Однако подход неплохо показал себя на многих номерах

In [60]:

# def text_from_numbers(image):
#     thresh_value = 100

#     image = cv2.resize(image, (256, 45))
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     image_gray = image * 255
#     _, thresh = cv2.threshold(image_gray, thresh_value, 255, cv2.THRESH_BINARY_INV)#90
    
#     cv2.imshow('thresh', thresh)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    
#     thresh = cv2.convertScaleAbs(thresh)
#     contours, _ =cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     if contours:
#         character_images = []
#         _, image_for_NN = cv2.threshold(image*255, thresh_value, 255, cv2.THRESH_BINARY)
#         for contour in contours:
#             x, y, w, h = cv2.boundingRect(contour)
#             if w>10 and h>10:#Попробуй уменьшить и увеличить
#                 y_max = y+h
#                 x_max = x+w

#                 x -= 10 #Может вместо этого добавлять вокруг белые пиксели?
#                 y -= 10
#                 x_max += 10
#                 y_max += 10

#                 x = max(0, x)
#                 y = max(0, y)
#                 x_max = min(x_max, image_for_NN.shape[1]-1)
#                 y_max = min(y_max, image_for_NN.shape[0]-1)

#                 symbol_for_NN = image_for_NN[y:y_max, x:x_max]
#                 #print(image_for_NN)
#                 # cv2.imshow('2212.jpg', symbol_for_NN)
#                 # cv2.waitKey(0)
#                 # cv2.destroyAllWindows()
#                 symbol_for_NN = cv2.resize(symbol_for_NN, (28,28))
#                 character_images.append(symbol_for_NN)
                        
#         #print(len(character_images))
        
#         predicted_chars = []
#         for char_image in character_images:
#             cv2.imshow('char_image', char_image)
#             cv2.waitKey(0)
#             cv2.destroyAllWindows()
#             char_image_tensor = torch.Tensor(char_image).unsqueeze(0).unsqueeze(0)
#             char_image_tensor = char_image_tensor.to(device)
#             pred = text_model(char_image_tensor)
#             real_char = symbols_keys[pred.argmax().item()]
#             print(pred.argmax().item(), real_char,'\n\n__________________________________________')
# Нужно переставить порядок символов, т.к. контуры идут не по порядку

In [61]:
#УЧЕСТЬ, что если ИИ распознает ту часть номера, где буквы, то если она распознает "0", то поменять "0" на "О" и наоборот
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def text_from_numbers(image):
    weight = 256
    height = 75
    image = cv2.resize(image, (weight, height))
    #image *= 255
    thresh_value = 90
    #_, image = cv2.threshold(image*255, thresh_value, 255, cv2.THRESH_BINARY)#90
    characters = []
    start = 18
    step = 29
    symb_number = 0
    for char in range(start, weight, step):
        # if symb_number > 7:
        #     break
        # if symb_number > 5:
        #     symbol = image[:height-17, char:char+step]
        # else:
        #     symbol = image[:, char:char+step]
        symbol = image[:, char-6:char+step+5]
        symbol = cv2.resize(symbol, (28, 28))
        characters.append(symbol)
        #print(symbol)
        #symb_number+=1
    characters = characters[:6]

    result = ''
    for symb in characters:
        # cv2.imshow('symbol', symb)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        symb = torch.tensor(symb).unsqueeze(0).unsqueeze(0)
        symb = symb.to(device)
        pred = text_model(symb)
        #print(pred)
        real_char = symbols_keys[pred.argmax().item()]
        result += real_char
        #print(pred.argmax().item(), real_char,'\n\n__________________________________________')

    print(result)
    cv2.imshow('i', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return result

In [62]:
text_from_numbers(test[0])

М090ЕТ


'М090ЕТ'